In [30]:
from misc_functions            import *
from db.Database import *
# import db.Area as Area
from db.AreaExit import *

from db.MudMap import MudMap
import networkx, json, os

In [31]:
def get_exits_from_area_id(area_id):
    exits = []
    try:
        exits = AreaExit.select().where((AreaExit.area_from_id == area_id))
    except AreaExit.DoesNotExist:
        exits = []
    return exits

area_id = 2
initial_exits = get_exits_from_area_id(area_id)

In [32]:
def get_exit_json(exits):
    # we wanna make some json edges and nodes/areas for cytoscape from our los map data
    # let's start simple and look out 2x steps from teh start area (Chapel areaid 2)
    edges = []

    for exit in initial_exits:
        # we should append json of the edge id, source (area_from_id), target (area_to_id)
        # skip if the area_to_id is null
        if exit.area_to_id is not None:
            edges.append({"id": exit.id, "source": exit.area_from_id, "source_name": exit.area_from.name, "target": exit.area_to_id, "target_name": exit.area_to.name})

    return edges

In [33]:
def get_areas_json(exits):
    areas = {}
    for exit in initial_exits:
        # we should append json of the edge id, source (area_from_id), target (area_to_id)
        # skip if the area_to_id is null
        areas[exit.area_from_id] = {"id": exit.area_from_id, "name": exit.area_from.name}
        
        if exit.area_to_id is not None:
            areas[exit.area_to_id] = {"id": exit.area_to_id, "name": exit.area_to.name}

    return areas

In [34]:
elements = []
nodes = []
edges = []
areas_mapped = [area_id]

def get_new_json(exits):
    nodes = []
    edges = []
    edges = get_exit_json(exits)
    areas = get_areas_json(exits)

    # for cytoscape, we need to have a list of nodes and a list of edges as json elements with data attributes
    for area in areas:
        nice_name = f'{areas[area]["id"]}_{areas[area]["name"]}' 
        nodes.append({"data": {"id": areas[area]["id"], "label": nice_name}, "position": {"x": 0, "y": 0}, "group": "nodes"})

    for edge in edges:
        print(edge)
        nice_name = f'{edge["id"]}_{edge["source_name"]}_{edge["target_name"]}'
        edges.append({"data": {"id": nice_name, "source": edge["source"], "target": edge["target"], "group": "edges"}})
    return nodes, edges

# starting from area id 2 let's recursively get all the exits and areas that are connected to it


nodes,edges = get_new_json(initial_exits)
for exit in initial_exits:
    if exit.area_to_id is not None and exit.area_to_id not in areas_mapped:
        area_id = exit.area_to_id
        exits = get_exits_from_area_id(area_id)
        temp_nodes,temp_edges = get_new_json(exits)
        nodes += temp_nodes
        edges += temp_edges
        areas_mapped.append(area_id)

elements = {"elements": {"nodes": nodes, "edges": edges},"style":[{"selector":"node","style":{"background-color":"rgb(102,102,102)","label":"data(label)"}},{"selector":"edge","style":{"width":"3px","line-color":"rgb(204,204,204)","target-arrow-color":"rgb(204,204,204)","target-arrow-shape":"triangle","curve-style":"bezier"}}],"data":{},"zoomingEnabled":True,"userZoomingEnabled":True,"zoom":1.337443680341475,"minZoom":1e-50,"maxZoom":1e+50,"panningEnabled":True,"userPanningEnabled":True,"pan":{"x":-182.43063789423763,"y":-155.91415698363767},"boxSelectionEnabled":True,"renderer":{"name":"canvas"}}

print(json.dumps(elements))

OperationalError: no such column: t1.x

In [ ]:
derp = MudMap()

OperationalError: no such column: t1.x

In [ ]:
print(json.dumps(derp.get_json()))

NameError: name 'derp' is not defined

In [ ]:
derp.los_map.remove_nodes_from(list(networkx.isolates(derp.los_map)))

In [ ]:
connected_areas = json.loads(json.dumps(derp.get_json()["elements"]))
print(connected_areas["nodes"])

[{'data': {'id': '2', 'value': 2, 'name': '2'}}, {'data': {'id': '3', 'value': 3, 'name': '3'}}, {'data': {'id': '4', 'value': 4, 'name': '4'}}, {'data': {'id': '5', 'value': 5, 'name': '5'}}, {'data': {'id': '6', 'value': 6, 'name': '6'}}, {'data': {'id': '7', 'value': 7, 'name': '7'}}, {'data': {'id': '8', 'value': 8, 'name': '8'}}, {'data': {'id': '9', 'value': 9, 'name': '9'}}, {'data': {'id': '10', 'value': 10, 'name': '10'}}, {'data': {'id': '11', 'value': 11, 'name': '11'}}, {'data': {'id': '12', 'value': 12, 'name': '12'}}, {'data': {'id': '13', 'value': 13, 'name': '13'}}, {'data': {'id': '14', 'value': 14, 'name': '14'}}, {'data': {'id': '15', 'value': 15, 'name': '15'}}, {'data': {'id': '16', 'value': 16, 'name': '16'}}, {'data': {'id': '17', 'value': 17, 'name': '17'}}, {'data': {'id': '18', 'value': 18, 'name': '18'}}, {'data': {'id': '19', 'value': 19, 'name': '19'}}, {'data': {'id': '20', 'value': 20, 'name': '20'}}, {'data': {'id': '21', 'value': 21, 'name': '21'}}, {'d

In [ ]:
for node in connected_areas["nodes"]:
    # get area id and then set the stuff
    area = Area.get_area_by_id(node["data"]["id"])
    node["data"]["label"] = str(area.id) + " " + area.name

In [ ]:
print(json.dumps(connected_areas))

{"nodes": [{"data": {"id": "2", "value": 2, "name": "2", "label": "2 The Chapel of Healing"}}, {"data": {"id": "3", "value": 3, "name": "3", "label": "3 Holly Lane"}}, {"data": {"id": "4", "value": 4, "name": "4", "label": "4 Holly Lane"}}, {"data": {"id": "5", "value": 5, "name": "5", "label": "5 Main Road / Holly Lane"}}, {"data": {"id": "6", "value": 6, "name": "6", "label": "6 Prayer Chamber"}}, {"data": {"id": "7", "value": 7, "name": "7", "label": "7 Main Road"}}, {"data": {"id": "8", "value": 8, "name": "8", "label": "8 Main Road"}}, {"data": {"id": "9", "value": 9, "name": "9", "label": "9 Grey Street / Main Road"}}, {"data": {"id": "10", "value": 10, "name": "10", "label": "10 Grey Street"}}, {"data": {"id": "11", "value": 11, "name": "11", "label": "11 Grey Street"}}, {"data": {"id": "12", "value": 12, "name": "12", "label": "12 Mindan Way / Grey Street"}}, {"data": {"id": "13", "value": 13, "name": "13", "label": "13 The Black Gate / Minden Way"}}, {"data": {"id": "14", "val

In [ ]:
# get the map_json.json file and read it
# if it doesn't exist do nothing

map_json = {}
if os.path.isfile('../notes/map_json.json'):
    with open('../notes/map_json.json', 'r') as f:
        map_json = json.load(f)
else:
    print("map_json.json doesn't exist")


In [ ]:
print(str(map_json["elements"]["nodes"][0]))

{'data': {'id': '2', 'value': 2, 'name': '2', 'label': '2 The Chapel of Healing'}, 'position': {'x': -206.36385244227378, 'y': -224.10895073737723}, 'group': 'nodes', 'removed': False, 'selected': False, 'selectable': True, 'locked': False, 'grabbable': True, 'pannable': False, 'classes': ''}
